In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
ds = load_iris()
X_train, X_test, y_train, y_test = train_test_split(ds['data'], ds['target'])

In [2]:
from ml_algos import LinearRegression
ml = LinearRegression()
ml.fit(X_train, y_train)

1.625
0.5722539016630676
0.403573248842362
0.3670234601418899
0.3506302010606957
0.3377014854751407
0.3257324523653402
0.31433134514846667
0.3034218177081979
0.2929752482139331
0.2829708518753786
0.2733897112559314
0.2642138577674907
0.2554261064771233
0.24701000427744893
0.23894979608999026
0.23123039485907398
0.22383735316890152
0.21675683611620905
0.20997559534079877
0.20348094415823342
0.1972607337469779
0.19130333034513725
0.18559759341396753
0.1801328547271675
0.17489889834670172
0.16988594144757005
0.16508461595553056
0.16048595096331103
0.15608135589230276
0.15186260436813295
0.14782181877984826
0.14395145549372845
0.1402442906939756
0.13669340682370199
0.13329217960076742
0.13003426558409226
0.12691359026711074
0.12392433667601446
0.12106093445138513
0.11831804939272322
0.11569057344624693
0.11317361511716913
0.1107624902884558
0.10845271342883282
0.10623998917353844
0.10412020426201865
0.1020894198174325
0.10014386395347535
0.09827992469464436
0.0964941431966562
0.09478320725

In [3]:
print(y_test)
print(ml.predict(X_test))

[2 2 0 1 0 0 2 0 2 2 1 0 0 2 1 1 2 2 1 2 2 0 1 1 1 1 0 0 0 0 0 2 2 2 2 1 1
 1]
[1.8609015049750448, 1.8493396765259864, 0.021076384726115952, 1.4963696006701348, 0.0019376226784869645, 0.017110003107479416, 1.6703436902012212, 0.04268386433584426, 1.9560687046622138, 1.481360079657237, 1.3602102202454252, 0.1081229090372812, 0.11565088422712458, 1.696380366418467, 1.038184144924505, 1.2166523736617492, 1.6708714477887419, 1.764211588508701, 1.1009401251970823, 1.6251547967883713, 1.7425576795321835, 0.07398369450947827, 1.369902570809331, 1.2824936773686777, 1.4375034618428921, 1.5031329477249982, 0.1656357299922304, 0.11017085436203794, 0.142984332782122, 0.11140521314308754, 0.0318801245309801, 1.8370897601152503, 1.5970233697131764, 1.68843853464847, 2.122575374986847, 1.1317636852338608, 1.4015067431710018, 1.0222536758513183]
